In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.options.display.max_colwidth = 30

# 체인 전체 통합 코드

In [39]:
votes = pd.read_csv('./complete_data/validator_votes.csv').drop(columns='Unnamed: 0')

chain_list = [
    'cosmos', 'kava', 'injective', 'axelar', 'osmosis', 'terra',
    'iris', 'dydx', 'secret', 'akash', 'stargaze', 'gravity-bridge',
    'sentinel', 'juno'
]
chain_dfs = {}

for chain_name in chain_list:
    chain_df = votes[votes[chain_name] == 1]
    relevant_columns = ['voter'] + [col for col in votes.columns if col.startswith(f"{chain_name}_")] + ['cluster_label']
    chain_df = chain_df[relevant_columns].reset_index(drop=True)
    
    chain_dfs[chain_name] = chain_df

# chain_dfs['cosmos']

/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3580/1375965733.py:1: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,61,63,64,65,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

In [40]:
for chain_name in chain_list:
    print(chain_name)

    cdf = chain_dfs[chain_name].copy()

    # 모든 열 값을 문자열로 변환 (NaN은 제외)
    cdf = cdf.applymap(lambda x: str(x) if pd.notnull(x) else x)

    # 데이터에 존재하는 key_labels만 추출
    key_labels = [label for label in ['YES', 'NO', 'NO_WITH_VETO', 'ABSTAIN', 'NO_VOTE'] if label in cdf.stack().unique()]

    # 열별로 YES, NO, NO_WITH_VETO, ABSTAIN, NO_VOTE, NaN의 개수를 계산
    first_df = cdf.apply(lambda col: col.value_counts(dropna=False)).fillna(0).T
    first_df = first_df.reindex(columns=key_labels, fill_value=0).astype(int)  # key_labels 기준 재정렬
    first_df = first_df.rename(columns={'NO_WITH_VETO': 'VETO'})

    print(f'{chain_name} first_df complete')

    # 첫 행에는 first_df에서 각 열의 최대값
    most_common_first = first_df.idxmax(axis=1)
    second_df_data = [most_common_first]

    # cluster_label별로 가장 많은 값 추가
    cluster_labels = cdf['cluster_label'].dropna().unique()

    for cluster in sorted(cluster_labels):
        cluster_data = cdf[cdf['cluster_label'] == str(cluster)]
        cluster_counts = cluster_data.apply(lambda col: col.value_counts(dropna=False)).fillna(0).T
        
        cluster_counts = cluster_counts.reindex(columns=key_labels, fill_value=0)  # 누락된 key_labels 보완
        most_common_cluster = cluster_counts.idxmax(axis=1)

        second_df_data.append(most_common_cluster)

    # # second_df 생성
    second_df = pd.DataFrame(second_df_data).drop(columns='cluster_label')
    second_df.index = ['Total'] + sorted(cluster_labels)

    print(f'{chain_name} second_df complete')
    
    # third_df 생성
    voters = cdf['voter']
    third_df_data = []

    for _, row in cdf.iterrows():
        voter = row['voter']
        cluster_label = row['cluster_label']
        
        # 각 투표 유형 개수 계산
        vote_counts = row.value_counts()
        yes_count = vote_counts.get('YES', 0)
        no_count = vote_counts.get('NO', 0)
        veto_count = vote_counts.get('NO_WITH_VETO', 0)
        abstain_count = vote_counts.get('ABSTAIN', 0)
        no_vote_count = vote_counts.get('NO_VOTE', 0)
        
        # 유효 투표와 전체 투표 계산
        valid_votes = yes_count + no_count + veto_count + abstain_count
        total_votes = valid_votes + no_vote_count

        # proposal 개수 계산
        proposal_num = len([col for col in row.index if col.startswith(f'{chain_name}_')])
        
        # second_df와 비교하여 매칭 계산
        overall_match = 0
        cluster_match = 0

        for proposal_id in row.index:
            if proposal_id.startswith(f'{chain_name}_'):
                proposal_vote = row[proposal_id]
                total_vote = second_df.loc['Total', proposal_id]

                # overall_match 계산
                if proposal_vote == total_vote:
                    overall_match += 1

                # cluster_match 계산
                if cluster_label in second_df.index:
                    cluster_vote = second_df.loc[cluster_label, proposal_id]
                    if proposal_vote == cluster_vote:
                        cluster_match += 1

        # 검증인 데이터 추가
        third_df_data.append({
            'voter': voter,
            'yes': yes_count,
            'no': no_count,
            'veto': veto_count,
            'abstain': abstain_count,
            'no_vote': no_vote_count,
            'valid_votes': valid_votes,
            'total_votes': total_votes,
            'proposal_num': proposal_num,
            'overall_match': overall_match,
            'cluster_match': cluster_match
        })

    # third_df 생성
    third_df = pd.DataFrame(third_df_data)

    # overall_match_rate, cluster_match_rate, participation_rate 계산
    third_df['overall_match_rate'] = third_df['overall_match'] / third_df['total_votes']
    third_df['cluster_match_rate'] = third_df['cluster_match'] / third_df['total_votes']
    third_df['participation_rate'] = third_df['valid_votes'] / third_df['proposal_num']

    print(f'{chain_name} third_df complete')

    data_without_cluster = chain_dfs[chain_name].drop(columns=['cluster_label'])
    voters = data_without_cluster['voter']
    data_without_cluster.set_index('voter', inplace=True)
    fourth_df = pd.DataFrame(index=voters, columns=voters)
    proposal_columns = [col for col in data_without_cluster.columns if col.startswith(f'{chain_name}_')]

    for voter_a in voters:
        for voter_b in voters:
            if voter_a == voter_b:
                fourth_df.loc[voter_a, voter_b] = 1.0
            else:
                matches = (data_without_cluster.loc[voter_a, proposal_columns] == 
                        data_without_cluster.loc[voter_b, proposal_columns]).sum()
                total_proposals = len(proposal_columns)
                match_percentage = matches / total_proposals if total_proposals > 0 else 0
                fourth_df.loc[voter_a, voter_b] = match_percentage
    fourth_df = fourth_df.astype(float)

    print(f'{chain_name} fourth_df complete')

    combined_df = third_df.copy()
    combined_df = combined_df.merge(cdf[['voter', 'cluster_label']], on='voter', how='inner')
    similarity_columns = {col: f'{col}' for col in fourth_df.columns}
    fourth_df_renamed = fourth_df.rename(columns=similarity_columns)

    combined_df = combined_df.merge(fourth_df_renamed, left_on='voter', right_index=True)
    high_similarity = fourth_df.apply(lambda row: [voter for voter in row[row >= 0.6].index if voter != row.name], axis=1)
    combined_df['high_similarity_voters'] = high_similarity.values
    
    combined_df.to_json(f'./complete_data/validator_result_data/validator_result_{chain_name}.json', orient='records')
    combined_df.to_csv(f'./complete_data/validator_result_data/validator_result_{chain_name}.csv')
    print(f'{chain_name} save complete')

cosmos
cosmos first_df complete
cosmos second_df complete


/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3580/321719768.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cdf = cdf.applymap(lambda x: str(x) if pd.notnull(x) else x)


cosmos third_df complete
cosmos fourth_df complete
cosmos save complete
kava
kava first_df complete
kava second_df complete


/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3580/321719768.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cdf = cdf.applymap(lambda x: str(x) if pd.notnull(x) else x)


kava third_df complete
kava fourth_df complete
kava save complete
injective
injective first_df complete


...

gravity-bridge third_df complete
gravity-bridge fourth_df complete
gravity-bridge save complete
sentinel
sentinel first_df complete
sentinel second_df complete
sentinel third_df complete
sentinel fourth_df complete


/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3580/321719768.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cdf = cdf.applymap(lambda x: str(x) if pd.notnull(x) else x)


sentinel save complete
juno
juno first_df complete
juno second_df complete
juno third_df complete


/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3580/321719768.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cdf = cdf.applymap(lambda x: str(x) if pd.notnull(x) else x)


juno fourth_df complete
juno save complete


# 체인 하나에 대한 코드

In [41]:
votes = pd.read_csv('./complete_data/validator_votes.csv').drop(columns='Unnamed: 0')
votes

/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3580/3495828067.py:1: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,61,63,64,65,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

,voter,osmosis_1,osmosis,osmosis_3,osmosis_5,osmosis_7,osmosis_22,osmosis_24,osmosis_29,osmosis_39,osmosis_50,osmosis_51,osmosis_55,osmosis_57,osmosis_58,osmosis_62,osmosis_72,osmosis_74,osmosis_76,osmosis_81,osmosis_82,osmosis_84,osmosis_96,osmosis_109,osmosis_111,osmosis_114,osmosis_120,osmosis_123,osmosis_130,osmosis_132,osmosis_133,osmosis_134,osmosis_135,osmosis_140,osmosis_141,osmosis_144,osmosis_146,osmosis_148,osmosis_149,osmosis_151,osmosis_152,osmosis_153,osmosis_156,osmosis_159,osmosis_163,osmosis_165,osmosis_166,osmosis_169,osmosis_170,osmosis_171,osmosis_173,osmosis_178,osmosis_180,osmosis_188,osmosis_190,osmosis_191,osmosis_211,osmosis_306,juno_4,juno,stargaze_11,stargaze,stargaze_15,juno_16,akash_16,akash,stargaze_16,stargaze_17,juno_18,juno_19,juno_20,stargaze_50,akash_52,stargaze_57,stargaze_58,stargaze_59,cosmos_60,cosmos,cosmos_63,cosmos_69,juno_72,cosmos_72,cosmos_75,cosmos_76,cosmos_77,cosmos_78,cosmos_79,cosmos_80,cosmos_81,stargaze_81,cosmos_82,juno_83,cosmos_83,juno_85,cosmos_86,cosmos_89,cosmos_90,cosmos_93,cosmos_94,stargaze_94,cosmos_99,cosmos_101,stargaze_104,juno_125,juno_141,osmosis_162,akash_182,akash_185,akash_187,osmosis_194,osmosis_195,osmosis_199,osmosis_200,osmosis_204,osmosis_206,osmosis_207,osmosis_208,osmosis_209,osmosis_216,osmosis_220,akash_229,akash_232,osmosis_232,akash_233,akash_234,akash_236,osmosis_237,akash_242,akash_243,akash_244,akash_245,akash_246,akash_247,akash_250,osmosis_251,akash_252,akash_253,akash_254,akash_255,osmosis_256,akash_261,osmosis_261,osmosis_262,akash_263,akash_266,osmosis_267,akash_268,osmosis_270,osmosis_271,osmosis_273,juno_284,juno_292,osmosis_305,juno_307,osmosis_309,osmosis_315,osmosis_316,osmosis_323,osmosis_326,osmosis_339,osmosis_342,osmosis_343,osmosis_349,osmosis_350,osmosis_362,osmosis_365,osmosis_367,osmosis_372,osmosis_378,osmosis_386,osmosis_388,osmosis_389,osmosis_390,osmosis_400,osmosis_402,osmosis_403,osmosis_419,osmosis_420,osmosis_421,osmosis_436,osmosis_442,osmosis_444,osmosis_451,osmosis_457,osmosis_480,osmosis_482,osmosis_492,osmosis_497,osmosis_498,osmosis_519,osmosis_530,osmosis_561,osmosis_564,osmosis_593,osmosis_610,osmosis_620,osmosis_638,osmosis_639,osmosis_640,osmosis_641,osmosis_645,osmosis_652,osmosis_655,cosmos_687,osmosis_688,osmosis_694,osmosis_700,osmosis_706,osmosis_707,osmosis_712,osmosis_741,osmosis_742,osmosis_755,osmosis_764,osmosis_785,cosmos_787,cosmos_791,cosmos_793,cosmos_800,cosmos_811,cosmos_814,cosmos_818,cosmos_826,cosmos_827,cosmos_836,cosmos_839,cosmos_848,cosmos_851,cosmos_853,cosmos_856,cosmos_858,cosmos_861,cosmos_862,cosmos_865,cosmos_867,cosmos_868,cosmos_895,cosmos_899,cosmos_912,cosmos_916,cosmos_917,cosmos_946,cosmos_952,cosmos_954,cosmos_956,osmosis_276,osmosis_277,osmosis_320,osmosis_352,osmosis_376,osmosis_394,osmosis_406,osmosis_417,osmosis_464,dydx_6,dydx,juno_7,akash_9,akash_14,juno_15,akash_22,akash_23,akash_24,iris_24,iris,dydx_24,iris_25,iris_26,akash_26,akash_28,iris_28,axelar_29,axelar,juno_30,akash_31,stargaze_35,juno_39,iris_41,dydx_44,iris_47,dydx_47,iris_48,dydx_48,iris_52,juno_59,juno_60,akash_63,cosmos_68,cosmos_73,iris_73,iris_74,cosmos_74,iris_76,iris_77,iris_78,iris_79,iris_84,iris_89,juno_89,cosmos_95,cosmos_98,stargaze_101,cosmos_103,cosmos_104,stargaze_128,dydx_138,dydx_142,stargaze_144,dydx_144,dydx_154,cosmos_155,dydx_156,dydx_157,dydx_164,dydx_165,axelar_166,axelar_167,axelar_168,axelar_169,axelar_170,axelar_171,axelar_172,injective_173,injective,axelar_174,dydx_175,dydx_176,axelar_178,axelar_188,axelar_190,axelar_191,axelar_192,axelar_193,injective_193,axelar_194,axelar_196,akash_199,axelar_199,akash_200,stargaze_200,axelar_200,axelar_201,cosmos_202,axelar_204,stargaze_205,axelar_206,akash_206,axelar_207,akash_207,akash_208,axelar_208,axelar_209,stargaze_210,axelar_211,axelar_212,axelar_213,stargaze_213,axelar_215,akash_216,axelar_216,axelar_217,axelar_218,axelar_219,axelar_220,axelar_222,axelar_223,stargaze_223,injective_226,stargaze_226,akash_230,akash_235,axelar_239,axelar

In [42]:
chain_list = [
    'cosmos', 'kava', 'injective', 'axelar', 'osmosis', 'terra',
    'iris', 'dydx', 'secret', 'akash', 'stargaze', 'gravity-bridge',
    'sentinel', 'juno'
]

chain_dfs = {}

# Process each chain
for chain_name in chain_list:
    # Select rows where the chain column equals 1
    chain_df = votes[votes[chain_name] == 1]
    relevant_columns = ['voter'] + [col for col in votes.columns if col.startswith(f"{chain_name}_")] + ['cluster_label']
    chain_df = chain_df[relevant_columns].reset_index(drop=True)
    
    # Store the resulting DataFrame in the dictionary
    chain_dfs[chain_name] = chain_df

# Display the resulting DataFrame structure for one chain as an example
chain_dfs['cosmos']

,voter,cosmos_60,cosmos_63,cosmos_69,cosmos_72,cosmos_75,cosmos_76,cosmos_77,cosmos_78,cosmos_79,cosmos_80,cosmos_81,cosmos_82,cosmos_83,cosmos_86,cosmos_89,cosmos_90,cosmos_93,cosmos_94,cosmos_99,cosmos_101,cosmos_687,cosmos_787,cosmos_791,cosmos_793,cosmos_800,cosmos_811,cosmos_814,cosmos_818,cosmos_826,cosmos_827,cosmos_836,cosmos_839,cosmos_848,cosmos_851,cosmos_853,cosmos_856,cosmos_858,cosmos_861,cosmos_862,cosmos_865,cosmos_867,cosmos_868,cosmos_895,cosmos_899,cosmos_912,cosmos_916,cosmos_917,cosmos_946,cosmos_952,cosmos_954,cosmos_956,cosmos_68,cosmos_73,cosmos_74,cosmos_95,cosmos_98,cosmos_103,cosmos_104,cosmos_155,cosmos_202,cosmos_717,cosmos_796,cosmos_920,cosmos_922,cosmos_930,cosmos_52,cosmos_42,cosmos_44,cosmos_45,cluster_label
0,Chainflow,ABSTAIN,YES,NO,NO,YES,YES,YES,ABSTAIN,YES,NO,NO,NO,NO,NO,YES,ABSTAIN,YES,YES,YES,YES,YES,ABSTAIN,ABSTAIN,ABSTAIN,ABSTAIN,NO,YES,NO,YES,ABSTAIN,YES,NO,YES,YES,ABSTAIN,NO,YES,YES,YES,YES,YES,ABSTAIN,ABSTAIN,YES,ABSTAIN,NO,YES,ABSTAIN,ABSTAIN,YES,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,Imperatorco,ABSTAIN,NaN,NO,YES,ABSTAIN,NO,YES,NO,ABSTAIN,NO,NO,NO_WITH_VETO,NO,ABSTAIN,NO,NO,ABSTAIN,YES,ABSTAIN,YES,YES,NO,NO,ABSTAIN,YES,NO,YES,NO,ABSTAIN,NO,NO,YES,WEIGHTED_VOTE,YES,NO,NO,YES,ABSTAIN,YES,ABSTAIN,YES,ABSTAIN,YES,ABSTAIN,YES,YES,YES,ABSTAIN,NO_WITH_VETO,ABSTAIN,NO,YES,NO_WITH_VETO,YES,ABSTAIN,ABSTAIN,NO,NO,NO,ABSTAIN,YES,ABSTAIN,ABSTAIN,YES,YES,NaN,NaN,NaN,NaN,3
2,StakeWithUs,NaN,YES,NO,ABSTAIN,ABSTAIN,ABSTAIN,YES,NO,YES,NO,NO,NO_WITH_VETO,NO,ABSTAIN,ABSTAIN,ABSTAIN,YES,YES,ABSTAIN,NaN,YES,ABSTAIN,NO,YES,YES,NO,YES,NO,YES,YES,ABSTAIN,ABSTAIN,YES,YES,YES,NO,NaN,YES,NaN,ABSTAIN,YES,YES,ABSTAIN,YES,ABSTAIN,NO,YES,NO,ABSTAIN,YES,NaN,YES,NO_WITH_VETO,YES,NaN,NaN,YES,NaN,ABSTAIN,NO,ABSTAIN,ABSTAIN,ABSTAIN,ABSTAIN,NaN,YES,NaN,NaN,NaN,3
3,Staking Fund,NaN,ABSTAIN,YES,ABSTAIN,ABSTAIN,NaN,YES,ABSTAIN,NO,NO,NO,NO_WITH_VETO,NO,NaN,YES,NO,ABSTAIN,YES,ABSTAIN,NO,NaN,NaN,NO,ABSTAIN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ABSTAIN,ABSTAIN,YES,YES,NO,YES,ABSTAIN,NO,NO,YES,NaN,NaN,NaN,NaN,NO,YES,YES,YES,7
4,BHarvest,NaN,YES,ABSTAIN,NaN,ABSTAIN,NO,YES,ABSTAIN,YES,NO,NO,NO_WITH_VETO,NO,NO,NO,NO,YES,YES,ABSTAIN,NO,YES,YES,NO,ABSTAIN,YES,NO,YES,NO,YES,YES,YES,YES,YES,YES,NO,NO,YES,NO,YES,YES,YES,NO,NO,YES,YES,NO,YES,NaN,NaN,NaN,NaN,YES,NO,YES,YES,NO,YES,ABSTAIN,YES,YES,YES,ABSTAIN,YES,ABSTAIN,NaN,NO,YES,YES,YES,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,CloudByte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,genznodes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO_VOTE,NO_VOTE,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,7
278,authztestvalidator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,2
279,NodeStake LottoDrop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ABSTAIN,NO_WITH_VETO,YES,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


In [43]:
chain_dfs['cosmos']

,voter,cosmos_60,cosmos_63,cosmos_69,cosmos_72,cosmos_75,cosmos_76,cosmos_77,cosmos_78,cosmos_79,cosmos_80,cosmos_81,cosmos_82,cosmos_83,cosmos_86,cosmos_89,cosmos_90,cosmos_93,cosmos_94,cosmos_99,cosmos_101,cosmos_687,cosmos_787,cosmos_791,cosmos_793,cosmos_800,cosmos_811,cosmos_814,cosmos_818,cosmos_826,cosmos_827,cosmos_836,cosmos_839,cosmos_848,cosmos_851,cosmos_853,cosmos_856,cosmos_858,cosmos_861,cosmos_862,cosmos_865,cosmos_867,cosmos_868,cosmos_895,cosmos_899,cosmos_912,cosmos_916,cosmos_917,cosmos_946,cosmos_952,cosmos_954,cosmos_956,cosmos_68,cosmos_73,cosmos_74,cosmos_95,cosmos_98,cosmos_103,cosmos_104,cosmos_155,cosmos_202,cosmos_717,cosmos_796,cosmos_920,cosmos_922,cosmos_930,cosmos_52,cosmos_42,cosmos_44,cosmos_45,cluster_label
0,Chainflow,ABSTAIN,YES,NO,NO,YES,YES,YES,ABSTAIN,YES,NO,NO,NO,NO,NO,YES,ABSTAIN,YES,YES,YES,YES,YES,ABSTAIN,ABSTAIN,ABSTAIN,ABSTAIN,NO,YES,NO,YES,ABSTAIN,YES,NO,YES,YES,ABSTAIN,NO,YES,YES,YES,YES,YES,ABSTAIN,ABSTAIN,YES,ABSTAIN,NO,YES,ABSTAIN,ABSTAIN,YES,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
1,Imperatorco,ABSTAIN,NaN,NO,YES,ABSTAIN,NO,YES,NO,ABSTAIN,NO,NO,NO_WITH_VETO,NO,ABSTAIN,NO,NO,ABSTAIN,YES,ABSTAIN,YES,YES,NO,NO,ABSTAIN,YES,NO,YES,NO,ABSTAIN,NO,NO,YES,WEIGHTED_VOTE,YES,NO,NO,YES,ABSTAIN,YES,ABSTAIN,YES,ABSTAIN,YES,ABSTAIN,YES,YES,YES,ABSTAIN,NO_WITH_VETO,ABSTAIN,NO,YES,NO_WITH_VETO,YES,ABSTAIN,ABSTAIN,NO,NO,NO,ABSTAIN,YES,ABSTAIN,ABSTAIN,YES,YES,NaN,NaN,NaN,NaN,3
2,StakeWithUs,NaN,YES,NO,ABSTAIN,ABSTAIN,ABSTAIN,YES,NO,YES,NO,NO,NO_WITH_VETO,NO,ABSTAIN,ABSTAIN,ABSTAIN,YES,YES,ABSTAIN,NaN,YES,ABSTAIN,NO,YES,YES,NO,YES,NO,YES,YES,ABSTAIN,ABSTAIN,YES,YES,YES,NO,NaN,YES,NaN,ABSTAIN,YES,YES,ABSTAIN,YES,ABSTAIN,NO,YES,NO,ABSTAIN,YES,NaN,YES,NO_WITH_VETO,YES,NaN,NaN,YES,NaN,ABSTAIN,NO,ABSTAIN,ABSTAIN,ABSTAIN,ABSTAIN,NaN,YES,NaN,NaN,NaN,3
3,Staking Fund,NaN,ABSTAIN,YES,ABSTAIN,ABSTAIN,NaN,YES,ABSTAIN,NO,NO,NO,NO_WITH_VETO,NO,NaN,YES,NO,ABSTAIN,YES,ABSTAIN,NO,NaN,NaN,NO,ABSTAIN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ABSTAIN,ABSTAIN,YES,YES,NO,YES,ABSTAIN,NO,NO,YES,NaN,NaN,NaN,NaN,NO,YES,YES,YES,7
4,BHarvest,NaN,YES,ABSTAIN,NaN,ABSTAIN,NO,YES,ABSTAIN,YES,NO,NO,NO_WITH_VETO,NO,NO,NO,NO,YES,YES,ABSTAIN,NO,YES,YES,NO,ABSTAIN,YES,NO,YES,NO,YES,YES,YES,YES,YES,YES,NO,NO,YES,NO,YES,YES,YES,NO,NO,YES,YES,NO,YES,NaN,NaN,NaN,NaN,YES,NO,YES,YES,NO,YES,ABSTAIN,YES,YES,YES,ABSTAIN,YES,ABSTAIN,NaN,NO,YES,YES,YES,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,CloudByte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
277,genznodes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NO_VOTE,NO_VOTE,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,7
278,authztestvalidator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,NaN,NaN,NaN,NaN,2
279,NodeStake LottoDrop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ABSTAIN,NO_WITH_VETO,YES,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7


In [44]:
import pandas as pd

# 데이터프레임 준비 (데이터 정리)
chain_dfs_cosmos = chain_dfs['cosmos'].copy()

# 모든 열 값을 문자열로 변환 (NaN은 제외)
chain_dfs_cosmos = chain_dfs_cosmos.applymap(lambda x: str(x) if pd.notnull(x) else x)

# 데이터에 존재하는 key_labels만 추출
key_labels = [label for label in ['YES', 'NO', 'NO_WITH_VETO', 'ABSTAIN', 'NO_VOTE'] if label in chain_dfs_cosmos.stack().unique()]

# 열별로 YES, NO, NO_WITH_VETO, ABSTAIN, NO_VOTE, NaN의 개수를 계산
first_df = chain_dfs_cosmos.apply(lambda col: col.value_counts(dropna=False)).fillna(0).T
first_df = first_df.reindex(columns=key_labels, fill_value=0).astype(int)  # key_labels 기준 재정렬
first_df = first_df.rename(columns={'NO_WITH_VETO': 'VETO'})
# 첫 행에는 first_df에서 각 열의 최대값
most_common_first = first_df.idxmax(axis=1)
second_df_data = [most_common_first]

# cluster_label별로 가장 많은 값 추가
cluster_labels = chain_dfs_cosmos['cluster_label'].dropna().unique()

for cluster in sorted(cluster_labels):
    cluster_data = chain_dfs_cosmos[chain_dfs_cosmos['cluster_label'] == str(cluster)]
    cluster_counts = cluster_data.apply(lambda col: col.value_counts(dropna=False)).fillna(0).T
    
    cluster_counts = cluster_counts.reindex(columns=key_labels, fill_value=0)  # 누락된 key_labels 보완
    most_common_cluster = cluster_counts.idxmax(axis=1)

    second_df_data.append(most_common_cluster)

# # second_df 생성
second_df = pd.DataFrame(second_df_data).drop(columns='cluster_label')
second_df.index = ['Total'] + sorted(cluster_labels)

/var/folders/nc/dhlk15l910g4hrbcq92y1cy40000gn/T/ipykernel_3580/627698355.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  chain_dfs_cosmos = chain_dfs_cosmos.applymap(lambda x: str(x) if pd.notnull(x) else x)


In [45]:
first_df

,YES,NO,VETO,ABSTAIN,NO_VOTE
voter,0,0,0,0,0
cosmos_60,24,4,0,28,40
cosmos_63,57,4,0,27,24
cosmos_69,27,62,8,14,18
cosmos_72,80,14,1,13,19
...,...,...,...,...,...
cosmos_52,15,18,0,4,50
cosmos_42,33,8,0,2,45
cosmos_44,43,2,0,1,44
cosmos_45,44,0,0,0,47


In [46]:
second_df

,voter,cosmos_60,cosmos_63,cosmos_69,cosmos_72,cosmos_75,cosmos_76,cosmos_77,cosmos_78,cosmos_79,cosmos_80,cosmos_81,cosmos_82,cosmos_83,cosmos_86,cosmos_89,cosmos_90,cosmos_93,cosmos_94,cosmos_99,cosmos_101,cosmos_687,cosmos_787,cosmos_791,cosmos_793,cosmos_800,cosmos_811,cosmos_814,cosmos_818,cosmos_826,cosmos_827,cosmos_836,cosmos_839,cosmos_848,cosmos_851,cosmos_853,cosmos_856,cosmos_858,cosmos_861,cosmos_862,cosmos_865,cosmos_867,cosmos_868,cosmos_895,cosmos_899,cosmos_912,cosmos_916,cosmos_917,cosmos_946,cosmos_952,cosmos_954,cosmos_956,cosmos_68,cosmos_73,cosmos_74,cosmos_95,cosmos_98,cosmos_103,cosmos_104,cosmos_155,cosmos_202,cosmos_717,cosmos_796,cosmos_920,cosmos_922,cosmos_930,cosmos_52,cosmos_42,cosmos_44,cosmos_45
Total,YES,NO_VOTE,YES,NO,YES,YES,NO,YES,NO,YES,NO,NO,YES,NO,ABSTAIN,NO,NO,YES,YES,ABSTAIN,ABSTAIN,YES,ABSTAIN,NO,YES,YES,NO,YES,NO,YES,NO,NO,YES,YES,YES,YES,NO,YES,NO,YES,YES,YES,NO,YES,YES,YES,NO,YES,YES,ABSTAIN,YES,NO,YES,VETO,YES,YES,NO,YES,YES,ABSTAIN,YES,YES,ABSTAIN,ABSTAIN,ABSTAIN,YES,NO_VOTE,NO_VOTE,NO_VOTE,NO_VOTE
0,YES,NO_VOTE,YES,NO,YES,NO,YES,YES,ABSTAIN,YES,NO,NO,YES,NO,ABSTAIN,YES,YES,YES,YES,NO,NO,YES,ABSTAIN,NO,YES,YES,YES,YES,YES,YES,NO,YES,YES,YES,YES,YES,NO,YES,NO,YES,YES,YES,NO,YES,YES,YES,YES,YES,NO,NO,YES,NO,YES,NO_WITH_VETO,YES,YES,NO,YES,YES,NO,YES,YES,NO,ABSTAIN,YES,YES,NO_VOTE,NO_VOTE,NO_VOTE,NO_VOTE
1,YES,NO_VOTE,NO_VOTE,YES,YES,NO_VOTE,NO_VOTE,NO_VOTE,NO_WITH_VETO,YES,NO,NO,YES,NO,NO_VOTE,ABSTAIN,NO_VOTE,NO_VOTE,NO_VOTE,YES,YES,YES,YES,YES,YES,YES,YES,YES,NO,YES,YES,YES,ABSTAIN,YES,YES,YES,NO,YES,NO,YES,YES,YES,YES,YES,YES,YES,YES,YES,NO_VOTE,NO,ABSTAIN,NO_VOTE,NO_VOTE,NO_VOTE,NO_VOTE,YES,NO,YES,YES,NO_VOTE,NO,YES,ABSTAIN,YES,ABSTAIN,YES,NO_VOTE,NO_VOTE,NO_VOTE,NO_VOTE
2,YES,ABSTAIN,YES,NO,YES,ABSTAIN,YES,YES,NO,YES,YES,YES,NO_WITH_VETO,YES,YES,NO,NO,YES,YES,YES,ABSTAIN,YES,YES,YES,YES,YES,NO,YES,YES,YES,NO,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,NO,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,NO,YES,YES,NO,YES,YES,YES,YES,YES,YES,YES,YES,YES,NO_VOTE,YES,YES,YES
3,YES,ABSTAIN,YES,NO,YES,YES,NO,YES,NO,YES,NO,NO,YES,NO,ABSTAIN,NO,NO,YES,YES,ABSTAIN,ABSTAIN,YES,ABSTAIN,NO,YES,YES,NO,YES,NO,YES,NO,NO,YES,YES,YES,YES,NO,YES,NO,YES,YES,YES,NO,YES,YES,YES,NO,YES,YES,ABSTAIN,YES,NO,YES,NO_WITH_VETO,YES,YES,NO,YES,YES,ABSTAIN,YES,YES,ABSTAIN,ABSTAIN,ABSTAIN,YES,NO_VOTE,YES,YES,YES
4,YES,YES,YES,NO,YES,YES,NO,YES,NO_WITH_VETO,YES,ABSTAIN,YES,NO_WITH_VETO,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,NO,YES,YES,YES,YES,ABSTAIN,YES,YES,ABSTAIN,ABSTAIN,YES,YES,NO,NO,YES,NO,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,ABSTAIN,YES,NO,YES,YES,YES,YES,NO,YES,YES,YES,YES,YES,YES,YES,ABSTAIN,YES,NO,ABSTAIN,YES,YES
5,YES,NO_VOTE,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,NO,YES,YES,YES,YES,YES,NO,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES,YES
6,YES,NO_VOTE,YES,NO,YES,ABSTAIN,YES,YES,NO,YES,NO,NO,YES,NO,NO,ABSTAIN,ABSTAIN,YES,YES,YES,ABSTAIN,YES,ABSTAIN,NO,YES,YES,NO,YES,NO,ABSTAIN,NO,ABSTAIN,YES,ABSTAIN,YES,YES,NO,YES,NO,YES,YES,YES,NO,ABSTAIN,YES,YES,NO,YES,ABSTAIN,ABSTAIN,YES,ABSTAIN,YES,NO_WITH_VETO,YES,YES,NO,YES,YES,NO,YES,YES,NO,ABSTAIN,ABSTAIN,YES,NO_VOTE,NO_VOTE,NO_VOTE,NO_VOTE
7,YES,NO_VOTE,YES,NO,YES,YES,NO,YES,NO,YES,NO,NO,YES,NO,ABSTAIN,NO,NO,YES,YES,ABSTAIN,ABSTAIN,YES,YES,NO,YES,YES,NO,YES,NO,YES,NO,NO,YES,YES,YES,YES,NO,YES,NO,YES,YES,YES,NO,YES,YES,YES,NO,YES,YES,ABSTAIN,YES,NO,YES,NO_WITH_VETO,YES,YES,NO,YES,YES,ABSTAIN,YES,YES,YES,ABSTAIN,ABSTAIN,YES,NO_VOTE,NO_VOTE,NO_VOTE,NO_VOTE


In [47]:
import pandas as pd

# third_df 생성
voters = chain_dfs_cosmos['voter']
third_df_data = []

for _, row in chain_dfs_cosmos.iterrows():
    voter = row['voter']
    cluster_label = row['cluster_label']
    
    # 각 투표 유형 개수 계산
    vote_counts = row.value_counts()
    yes_count = vote_counts.get('YES', 0)
    no_count = vote_counts.get('NO', 0)
    veto_count = vote_counts.get('NO_WITH_VETO', 0)
    abstain_count = vote_counts.get('ABSTAIN', 0)
    no_vote_count = vote_counts.get('NO_VOTE', 0)
    
    # 유효 투표와 전체 투표 계산
    valid_votes = yes_count + no_count + veto_count + abstain_count
    total_votes = valid_votes + no_vote_count

    # proposal 개수 계산
    proposal_num = len([col for col in row.index if col.startswith('cosmos_')])
    
    # second_df와 비교하여 매칭 계산
    overall_match = 0
    cluster_match = 0

    for proposal_id in row.index:
        if proposal_id.startswith('cosmos_'):
            proposal_vote = row[proposal_id]
            total_vote = second_df.loc['Total', proposal_id]

            # overall_match 계산
            if proposal_vote == total_vote:
                overall_match += 1

            # cluster_match 계산
            if cluster_label in second_df.index:
                cluster_vote = second_df.loc[cluster_label, proposal_id]
                if proposal_vote == cluster_vote:
                    cluster_match += 1

    # 검증인 데이터 추가
    third_df_data.append({
        'voter': voter,
        'yes': yes_count,
        'no': no_count,
        'veto': veto_count,
        'abstain': abstain_count,
        'no_vote': no_vote_count,
        'valid_votes': valid_votes,
        'total_votes': total_votes,
        'proposal_num': proposal_num,
        'overall_match': overall_match,
        'cluster_match': cluster_match
    })

# third_df 생성
third_df = pd.DataFrame(third_df_data)

# overall_match_rate, cluster_match_rate, participation_rate 계산
third_df['overall_match_rate'] = third_df['overall_match'] / third_df['total_votes']
third_df['cluster_match_rate'] = third_df['cluster_match'] / third_df['total_votes']
third_df['participation_rate'] = third_df['valid_votes'] / third_df['proposal_num']

# 결과 출력
third_df

,voter,yes,no,veto,abstain,no_vote,valid_votes,total_votes,proposal_num,overall_match,cluster_match,overall_match_rate,cluster_match_rate,participation_rate
0,Chainflow,24,13,0,14,0,51,51,69,29,30,0.568627,0.588235,0.739130
1,Imperatorco,21,20,3,19,0,63,63,69,37,39,0.587302,0.619048,0.913043
2,StakeWithUs,24,12,2,19,0,57,57,69,38,39,0.666667,0.684211,0.826087
3,Staking Fund,12,11,1,10,0,34,34,69,14,14,0.411765,0.411765,0.492754
4,BHarvest,33,20,1,8,0,62,62,69,42,45,0.677419,0.725806,0.898551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,CloudByte,1,0,0,0,0,1,1,69,1,1,1.000000,1.000000,0.014493
277,genznodes,2,1,0,0,2,3,5,69,3,3,0.600000,0.600000,0.043478
278,authztestvalidator,1,0,0,0,0,1,1,69,1,1,1.000000,1.000000,0.014493
279,NodeStake LottoDrop,1,1,1,1,0,4,4,69,2,2,0.500000,0.500000,0.057971


In [48]:
data_without_cluster = chain_dfs['cosmos'].drop(columns=['cluster_label'])
voters = data_without_cluster['voter']
data_without_cluster.set_index('voter', inplace=True)
fourth_df = pd.DataFrame(index=voters, columns=voters)
proposal_columns = [col for col in data_without_cluster.columns if col.startswith('cosmos_')]

for voter_a in voters:
    for voter_b in voters:
        if voter_a == voter_b:
            fourth_df.loc[voter_a, voter_b] = 1.0
        else:
            matches = (data_without_cluster.loc[voter_a, proposal_columns] == 
                       data_without_cluster.loc[voter_b, proposal_columns]).sum()
            total_proposals = len(proposal_columns)
            match_percentage = matches / total_proposals if total_proposals > 0 else 0
            fourth_df.loc[voter_a, voter_b] = match_percentage
fourth_df = fourth_df.astype(float)

fourth_df

voter,Chainflow,Imperatorco,StakeWithUs,Staking Fund,BHarvest,Provalidator,Simply Staking,StakeLabzone,BouBouNode,SpacePotato,Crosnest,Vitwit Previously Witval,SG1,KalpaTech,Chorus One,Forbole,in3scom,vidoinfo,Swiss Staking,01node,stakesystems autocompound,kytzu,POSTHUMAN StakeDrop,Ping,Notional,jabbey,0basevc,Sr20de,Smart Stake,WhisperNode,openbitlab,Node Guardians,Nodeist,stakefish,Zero Knowledge Validator ZKV,Chill Validation,Figment,a41,CrowdControl,alphabai,Tessellated,Gelotto Slash Protected,Golden Ratio Staking,Stakewollecom 100 Insurance,Ubik Capital,Kahuna,CommunityStaking,Nocturnal Labs,decentralizehk DHK dao,Stakelyio,interchainFM,ECO Stake,NosNode,Citadelone,Informal Systems,LavenderFive Nodes,KingSuper,Frens,Cosmostation,Kleomedes,DOUBLETOP,White Whale,Silk Nodes,BlockHunters,GATA HUB,BasBlock,Kintsugi Nodes,NodesGuru,Coinage x DAIC,blockscape,Keplr,Stakecito,Meria,PRO Delegators,Interstellar Lounge,DELIGHT,DSRV,Blockdaemon,MMS,BronBro,Cypher Core,node101,test,Stakeflow,ChainLayer,Active Nodes,strangelove,Allnodes,Klub Staking,FreshSTAKING,SmartNodes,CryptoDungeon,commercionetwork,SHUTTING DOWN AUDITone,S16 Research Ventures,dForce,ShapeShift DAO,Leonoors Cryptoman,iqlusion,Tedcryptoio TedLotto,validatornetwork,Trust Nodes,Interbloc,Valnodes Carbon Neutral Staking,Everstake,CroutonDigital,StakeRelax,Vortexlive,Blockpower,AIR DROP STATION,IcyCRO,Inactive Validator,Sunshine Validation solarpower,Multiplex,PRYZM StakeDrop,EZ Staking,Regenerator Green Validator,Stakin,Disperze,windpowerstake,Please Redelegate,WeStaking,redelegateplease,Enigma,nysanetwork,ChainTools,Shutting down redelegate,HashKey Cloud,uGaenn,PUPMS,LOA Labs,Shutting down please redelegate,Wetez,Architect Nodes,BitValidator,Inactive,Huginn,Crypto Plant,Virtual Hive,Easy 2 Stake,Sikka,SNZPool,Huobi,stakezone,MANTRA,cosmosrescue,DLINODESCOM,decommissioned,Dora Factory PGS,polkachucom,Oni Stake for BoostDrop,0xDEADBEEF,w3coins,StakeSeeker by BTCS,P2PORG P2P Validator,TTT,coinhallorg,Atomic power node,HLT,DECOMMISSIONED,bitszn,hashtower,Valid8r,Stigga,Nodiums,Station8,Cosmic Validator 100 Uptime Autocompound,HelionCosmosValidator,The Chicken Coop NFT Airdrops,chainnode,test kim,Certus One,Colossus,Bunkerstakeio,Andromaverse Labs,Post Road,Twinstake,AuraStake,NASDAQBTCS,grantfish,GAME,deboREDELEGATE FROM US,noma,lunamint,Compass,Dawnsworld,IRISNET,Citizen Web3,omega3,BlueStake,CryptoCrew Validators Winner GameOfChains,securesecrets,zoomerlabs,Earn Stash,Kiln,Luganodes,HighStakesch Stake for Airdrop,AgoraNodes,DokiaCapital,Moonstake,Cosmos Spaces,Quokka Stake,airdropdao,Don Cryptonium,MultiChain ventures,Sygnum,ChainUp NewHuo,MathWallet,MyCosmosValidator,end,TeeNode,Fanfury furyfan Stake to Win,Paradigm,UNSTAKEREDELEGATE,InfStones,Flipside,Stargaze,kochacolaj,NO Fee to 2025 meleaxyz,DragonStake,DelegaNetworks,nyliranet,KysenPool Sky,DO NOT DELEGATE Defi Wallet,Umbrella,Sentinel,Staking Facilities,Crypto Lion,Blocks United blocksunitedcom,TerraBitcoin Club,Liqee,BlockPI,Stir,Lightning Capital Decommissioning May 2 Please Unbond,Binary Holdings,STAKE FRITES DO NOT DELEGATE,Coinbase Cloud,Numenorone,Sunflower,Cryptostakecom,inactive validator Do NOT delegate,decomm,PumpkinFarms,CEXIO Validator,Ztakeorg,Huobi1,stake2earn,nakanodoxyz,Onbloc Node,RoundTable21 by WildSage Labs,Earthstar,Moonlet,sleepykitten,JADE DAO,KuCoin,KEK,melange,King Hugo,Ivaldi Labs,anythings,HyperblocksPro,Yieldera,CROSSTECH,CLOSED DOWN,Amplifier,Commons Hub,Factor,bitlabs,Alessandro Mazza,kjnodescom,BigBrainStaking,Defi Wallet,Leap Wallet,cosmos Af,Altiplanic,Inu X,CloudByte,genznodes,authztestvalidator,NodeStake LottoDrop,biglazycatcom
voter,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Chainflow,1.000000,0.318841,0.405797,0.115942,0.391304,0.318841,0.40579

In [49]:
combined_df = third_df.copy()

similarity_columns = {col: f'{col}' for col in fourth_df.columns}
fourth_df_renamed = fourth_df.rename(columns=similarity_columns)

combined_df = combined_df.merge(fourth_df_renamed, left_on='voter', right_index=True)
high_similarity = fourth_df.apply(lambda row: [voter for voter in row[row >= 0.6].index if voter != row.name], axis=1)
combined_df['high_similarity_voters'] = high_similarity.values

combined_df

,voter,yes,no,veto,abstain,no_vote,valid_votes,total_votes,proposal_num,overall_match,cluster_match,overall_match_rate,cluster_match_rate,participation_rate,Chainflow,Imperatorco,StakeWithUs,Staking Fund,BHarvest,Provalidator,Simply Staking,StakeLabzone,BouBouNode,SpacePotato,Crosnest,Vitwit Previously Witval,SG1,KalpaTech,Chorus One,Forbole,in3scom,vidoinfo,Swiss Staking,01node,stakesystems autocompound,kytzu,POSTHUMAN StakeDrop,Ping,Notional,jabbey,0basevc,Sr20de,Smart Stake,WhisperNode,openbitlab,Node Guardians,Nodeist,stakefish,Zero Knowledge Validator ZKV,Chill Validation,Figment,a41,CrowdControl,alphabai,Tessellated,Gelotto Slash Protected,Golden Ratio Staking,Stakewollecom 100 Insurance,Ubik Capital,Kahuna,CommunityStaking,Nocturnal Labs,decentralizehk DHK dao,Stakelyio,interchainFM,ECO Stake,NosNode,Citadelone,Informal Systems,LavenderFive Nodes,KingSuper,Frens,Cosmostation,Kleomedes,DOUBLETOP,White Whale,Silk Nodes,BlockHunters,GATA HUB,BasBlock,Kintsugi Nodes,NodesGuru,Coinage x DAIC,blockscape,Keplr,Stakecito,Meria,PRO Delegators,Interstellar Lounge,DELIGHT,DSRV,Blockdaemon,MMS,BronBro,Cypher Core,node101,test,Stakeflow,ChainLayer,Active Nodes,strangelove,Allnodes,Klub Staking,FreshSTAKING,SmartNodes,CryptoDungeon,commercionetwork,SHUTTING DOWN AUDITone,S16 Research Ventures,dForce,ShapeShift DAO,Leonoors Cryptoman,iqlusion,Tedcryptoio TedLotto,validatornetwork,Trust Nodes,Interbloc,Valnodes Carbon Neutral Staking,Everstake,CroutonDigital,StakeRelax,Vortexlive,Blockpower,AIR DROP STATION,IcyCRO,Inactive Validator,Sunshine Validation solarpower,Multiplex,PRYZM StakeDrop,EZ Staking,Regenerator Green Validator,Stakin,Disperze,windpowerstake,Please Redelegate,WeStaking,redelegateplease,Enigma,nysanetwork,ChainTools,Shutting down redelegate,HashKey Cloud,uGaenn,PUPMS,LOA Labs,Shutting down please redelegate,Wetez,Architect Nodes,BitValidator,Inactive,Huginn,Crypto Plant,Virtual Hive,Easy 2 Stake,Sikka,SNZPool,Huobi,stakezone,MANTRA,cosmosrescue,DLINODESCOM,decommissioned,Dora Factory PGS,polkachucom,Oni Stake for BoostDrop,0xDEADBEEF,w3coins,StakeSeeker by BTCS,P2PORG P2P Validator,TTT,coinhallorg,Atomic power node,HLT,DECOMMISSIONED,bitszn,hashtower,Valid8r,Stigga,Nodiums,Station8,Cosmic Validator 100 Uptime Autocompound,HelionCosmosValidator,The Chicken Coop NFT Airdrops,chainnode,test kim,Certus One,Colossus,Bunkerstakeio,Andromaverse Labs,Post Road,Twinstake,AuraStake,NASDAQBTCS,grantfish,GAME,deboREDELEGATE FROM US,noma,lunamint,Compass,Dawnsworld,IRISNET,Citizen Web3,omega3,BlueStake,CryptoCrew Validators Winner GameOfChains,securesecrets,zoomerlabs,Earn Stash,Kiln,Luganodes,HighStakesch Stake for Airdrop,AgoraNodes,DokiaCapital,Moonstake,Cosmos Spaces,Quokka Stake,airdropdao,Don Cryptonium,MultiChain ventures,Sygnum,ChainUp NewHuo,MathWallet,MyCosmosValidator,end,TeeNode,Fanfury furyfan Stake to Win,Paradigm,UNSTAKEREDELEGATE,InfStones,Flipside,Stargaze,kochacolaj,NO Fee to 2025 meleaxyz,DragonStake,DelegaNetworks,nyliranet,KysenPool Sky,DO NOT DELEGATE Defi Wallet,Umbrella,Sentinel,Staking Facilities,Crypto Lion,Blocks United blocksunitedcom,TerraBitcoin Club,Liqee,BlockPI,Stir,Lightning Capital Decommissioning May 2 Please Unbond,Binary Holdings,STAKE FRITES DO NOT DELEGATE,Coinbase Cloud,Numenorone,Sunflower,Cryptostakecom,inactive validator Do NOT delegate,decomm,PumpkinFarms,CEXIO Validator,Ztakeorg,Huobi1,stake2earn,nakanodoxyz,Onbloc Node,RoundTable21 by WildSage Labs,Earthstar,Moonlet,sleepykitten,JADE DAO,KuCoin,KEK,melange,King Hugo,Ivaldi Labs,anythings,HyperblocksPro,Yieldera,CROSSTECH,CLOSED DOWN,Amplifier,Commons Hub,Factor,bitlabs,Alessandro Mazza,kjnodescom,BigBrainStaking,Defi Wallet,Leap Wallet,cosmos Af,Altiplanic,Inu X,CloudByte,genznodes,authztestvalidator,NodeStake LottoDrop,biglazycatcom,high_similarity_voters
0,Chainflow,24,13,0,14,0,51,51,69,29,30,0.568627,0.588235,0.739130,1.000000,0.318841,0.405797,0.115942,0.391304,0.318841,0.405797,0.289855,0.043478,0.000000,0.231884,0.420290,0.275362,

In [11]:
# combined_df.to_json('./complete_data/validator_result_data/test.json', orient='records')
# combined_df.to_csv('./complete_data/validator_result_data/test.csv')

In [12]:
adf = pd.read_csv('./complete_data/validator_result_data/validator_result_akash.csv').drop(columns='Unnamed: 0')
adf

,voter,yes,no,veto,abstain,no_vote,valid_votes,total_votes,proposal_num,overall_match,cluster_match,overall_match_rate,cluster_match_rate,participation_rate,cluster_label,Chainflow,Imperatorco,AutoStake Slash Protected,StakeWithUs,Simply Staking,StakeLabzone,SpacePotato,Vitwit Previously Witval,SG1,Chorus One,Forbole,POSTHUMAN StakeDrop,Ping,0basevc,Smart Stake,block pane,WhisperNode,a41,Stakyio,Kahuna,16psyche,Nocturnal Labs,decentralizehk DHK dao,Kalia Network,ECO Stake,NosNode,Citadelone,Informal Systems,LavenderFive Nodes,Frens,Cosmostation,Kleomedes,OmniFlix Network,GATA HUB,hydrogen18,Coinage x DAIC,Stakecito,Meria,BronBro,Cypher Core,ChainLayer,Active Nodes,DACM,strangelove,ushakov,c29r3,Allnodes,PoS Node,Tedcryptoio TedLotto,kooltek68,CroutonDigital,coaction,EZ Staking,Stakin,notional,WeStaking,SmartNodes Slash Protected,Yurbason,Neta DAO,Chandra Station,Stake Frites,Decloud Nodes Lab,HashKey Cloud,chainvibes,Consensus One,Stakewollecom Autocompound,Wetez,Architect Nodes,HEXXAGON,DECOMISSIONED AUDITone,Army IDs,Bit Cat,Easy 2 Stake,Nodeasycom,jetlife,Bliss Dynamics,Sikka,PLEASE REDELEGATECLOSED,SNZPool,Autonomy Formerly PrithviDevs,Baset,Huobi,ChainodeTech,kano,Cosmic Validator Auto Compound,ValidatorNode,UNDELEGATE FROM HERE,stakezone,gunray,Bi23,Psycho Node,Bitoven,Komichaincom,BoxedCloudnet,MANTRA,eol,cosmosrescue,SkyNet Validators,Cosmonaut Stakes AKT,Dwebnet,DLINODESCOM,GlacierLuo,Domination Finance,Arcturian Tech,Akash Af,ALTER pathrocknetwork,decommissioned,BridgeTower,Dora Factory PGS,MedasDigital,STAKINOLD,Cloudmos,Former Moultrie Audits Inactive,MOVING TO NAKANODOXYZ,VaultStaking,TRGC By Liquify,astravoyage,Anonstake,polkachucom,kavalabs,AGE Network,acheron,Octopos,shepaintedfireacrossthesky,monolith,SHEA Staking,Dorminik,UNBONDING,Inactive pls undelegate,GreenStake One,inactive,Oni Stake for BoostDrop,Praetor App,0xDEADBEEF,w3coins,Vixello,Aurora Staking,50 Validator Airdrop,Passage,NoDumpNode,Jormungand 0y,INACTIVE Please Redelegate Trivium TriviumNetwork,StakeSeeker by BTCS,Crypto Lab Tech,europlots,CryptoCrew Validators,Quasarch,Ariel Akash Insider Powered by NextNetWorks,Atomstaking,black lodge,SHUTTING DOWN,Crypto and Coffee,FoundryUSA,OGnome,Defi Dojo Validation,high_similarity_voters
0,Chainflow,20,1,0,4,0,25,25,50,20,21,0.800000,0.840000,0.50,3,1.00,0.24,0.28,0.32,0.28,0.12,0.20,0.30,0.06,0.38,0.22,0.34,0.14,0.40,0.12,0.00,0.28,0.0,0.00,0.12,0.28,0.36,0.30,0.38,0.36,0.30,0.28,0.26,0.34,0.06,0.40,0.18,0.22,0.26,0.02,0.14,0.36,0.36,0.0,0.36,0.28,0.36,0.18,0.20,0.02,0.32,0.32,0.36,0.32,0.26,0.04,0.00,0.26,0.28,0.08,0.28,0.02,0.0,0.34,0.12,0.34,0.36,0.04,0.38,0.02,0.36,0.04,0.00,0.0,0.00,0.42,0.10,0.24,0.28,0.04,0.32,0.02,0.02,0.02,0.20,0.00,0.04,0.24,0.00,0.32,0.38,0.02,0.04,0.00,0.24,0.02,0.24,0.34,0.08,0.0,0.00,0.06,0.04,0.28,0.02,0.00,0.00,0.02,0.04,0.12,0.12,0.0,0.24,0.30,0.0,0.02,0.38,0.00,0.02,0.02,0.12,0.02,0.38,0.38,0.06,0.06,0.00,0.00,0.0,0.00,0.00,0.20,0.00,0.06,0.00,0.00,0.20,0.36,0.00,0.28,0.16,0.22,0.32,0.00,0.32,0.22,0.00,0.10,0.20,0.34,0.00,0.34,0.12,0.34,0.04,0.0,0.16,0.22,0.02,0.04,[]
1,Imperatorco,23,4,1,21,0,49,49,50,25,28,0.510204,0.571429,0.98,3,0.24,1.00,0.72,0.52,0.48,0.22,0.28,0.36,0.12,0.48,0.42,0.38,0.08,0.54,0.42,0.12,0.20,0.0,0.02,0.12,0.32,0.44,0.38,0.48,0.58,0.46,0.66,0.40,0.40,0.04,0.62,0.40,0.42,0.24,0.02,0.14,0.40,0.40,0.0,0.38,0.50,0.52,0.26,0.20,0.06,0.36,0.42,0.46,0.26,0.24,0.04,0.04,0.24,0.62,0.30,0.42,0.00,0.0,0.42,0.22,0.44,0.44,0.04,0.50,0.00,0.52,0.16,0.04,0.0,0.04,0.50,0.20,0.34,0.28,0.10,0.44,0.02,0.06,0.02,0.28,0.02,0.06,0.26,0.02,0.58,0.48,0.00,0.08,0.02,0.34,0.08,0.56,0.42,0.08,0.0,0.06,0.02,0.10,0.30,0.02,0.02,0.02,0.00,0.04,0.10,0.10,0.0,0.34,0.24,0.0,0.06,0.50,0.06,0.04,0.06,0.22,0.00,0.42,0.50,0.12,0.08,0.02,0.02,0.0,0.02,0.02,0.14,0.02,0.24,0.02,0.02,0.16,0.44,0.00,0.48,0.14,0.18,0.42,0.04,0.46,0.26,0.02,0.12,0.16,0.36,0.08,0.34,0.04,0.32,0.06,0.0,0.10,0.14,0.00,0.04,[' AutoStake Slash Protec...
2,AutoStake Slash Protected,34,5,1,10,0,50,50,50,36,37,0.7

In [4]:
adf[adf['voter']=='SG1']

,voter,yes,no,veto,abstain,no_vote,valid_votes,total_votes,proposal_num,overall_match,cluster_match,overall_match_rate,cluster_match_rate,participation_rate,Chainflow,Imperatorco,AutoStake Slash Protected,StakeWithUs,Simply Staking,StakeLabzone,SpacePotato,Vitwit Previously Witval,SG1,Chorus One,Forbole,POSTHUMAN StakeDrop,Ping,0basevc,Smart Stake,block pane,WhisperNode,a41,Stakyio,Kahuna,16psyche,Nocturnal Labs,decentralizehk DHK dao,Kalia Network,ECO Stake,NosNode,Citadelone,Informal Systems,LavenderFive Nodes,Frens,Cosmostation,Kleomedes,OmniFlix Network,GATA HUB,hydrogen18,Coinage x DAIC,Stakecito,Meria,BronBro,Cypher Core,ChainLayer,Active Nodes,DACM,strangelove,ushakov,c29r3,Allnodes,PoS Node,Tedcryptoio TedLotto,kooltek68,CroutonDigital,coaction,EZ Staking,Stakin,notional,WeStaking,SmartNodes Slash Protected,Yurbason,Neta DAO,Chandra Station,Stake Frites,Decloud Nodes Lab,HashKey Cloud,chainvibes,Consensus One,Stakewollecom Autocompound,Wetez,Architect Nodes,HEXXAGON,DECOMISSIONED AUDITone,Army IDs,Bit Cat,Easy 2 Stake,Nodeasycom,jetlife,Bliss Dynamics,Sikka,PLEASE REDELEGATECLOSED,SNZPool,Autonomy Formerly PrithviDevs,Baset,Huobi,ChainodeTech,kano,Cosmic Validator Auto Compound,ValidatorNode,UNDELEGATE FROM HERE,stakezone,gunray,Bi23,Psycho Node,Bitoven,Komichaincom,BoxedCloudnet,MANTRA,eol,cosmosrescue,SkyNet Validators,Cosmonaut Stakes AKT,Dwebnet,DLINODESCOM,GlacierLuo,Domination Finance,Arcturian Tech,Akash Af,ALTER pathrocknetwork,decommissioned,BridgeTower,Dora Factory PGS,MedasDigital,STAKINOLD,Cloudmos,Former Moultrie Audits Inactive,MOVING TO NAKANODOXYZ,VaultStaking,TRGC By Liquify,astravoyage,Anonstake,polkachucom,kavalabs,AGE Network,acheron,Octopos,shepaintedfireacrossthesky,monolith,SHEA Staking,Dorminik,UNBONDING,Inactive pls undelegate,GreenStake One,inactive,Oni Stake for BoostDrop,Praetor App,0xDEADBEEF,w3coins,Vixello,Aurora Staking,50 Validator Airdrop,Passage,NoDumpNode,Jormungand 0y,INACTIVE Please Redelegate Trivium TriviumNetwork,StakeSeeker by BTCS,Crypto Lab Tech,europlots,CryptoCrew Validators,Quasarch,Ariel Akash Insider Powered by NextNetWorks,Atomstaking,black lodge,SHUTTING DOWN,Crypto and Coffee,FoundryUSA,OGnome,Defi Dojo Validation,high_similarity_voters
8,SG1,6,2,0,6,0,14,14,50,6,7,0.428571,0.5,0.28,0.06,0.12,0.14,0.08,0.06,0.1,0.08,0.08,1.0,0.12,0.1,0.1,0.02,0.12,0.14,0.04,0.02,0.0,0.0,0.0,0.06,0.08,0.08,0.1,0.14,0.08,0.16,0.06,0.1,0.0,0.12,0.1,0.14,0.06,0.02,0.0,0.1,0.08,0.0,0.1,0.1,0.08,0.08,0.02,0.02,0.08,0.08,0.12,0.06,0.06,0.0,0.02,0.08,0.08,0.06,0.12,0.0,0.0,0.08,0.06,0.12,0.08,0.02,0.12,0.02,0.1,0.06,0.0,0.0,0.0,0.14,0.06,0.08,0.08,0.02,0.12,0.0,0.04,0.0,0.12,0.0,0.04,0.08,0.0,0.1,0.14,0.0,0.04,0.0,0.08,0.04,0.1,0.12,0.06,0.0,0.02,0.0,0.02,0.04,0.0,0.02,0.02,0.0,0.0,0.0,0.04,0.0,0.1,0.04,0.0,0.02,0.1,0.0,0.02,0.02,0.08,0.0,0.08,0.1,0.04,0.02,0.0,0.0,0.0,0.0,0.0,0.02,0.02,0.1,0.02,0.02,0.02,0.12,0.0,0.12,0.02,0.0,0.1,0.0,0.1,0.08,0.0,0.06,0.04,0.06,0.0,0.08,0.0,0.06,0.02,0.0,0.0,0.02,0.0,0.0,[]


In [7]:
adf = pd.read_json('./complete_data/validator_result_data/validator_result_akash.json')

In [8]:
adf

,voter,yes,no,veto,abstain,no_vote,valid_votes,total_votes,proposal_num,overall_match,cluster_match,overall_match_rate,cluster_match_rate,participation_rate,Chainflow,Imperatorco,AutoStake Slash Protected,StakeWithUs,Simply Staking,StakeLabzone,SpacePotato,Vitwit Previously Witval,SG1,Chorus One,Forbole,POSTHUMAN StakeDrop,Ping,0basevc,Smart Stake,block pane,WhisperNode,a41,Stakyio,Kahuna,16psyche,Nocturnal Labs,decentralizehk DHK dao,Kalia Network,ECO Stake,NosNode,Citadelone,Informal Systems,LavenderFive Nodes,Frens,Cosmostation,Kleomedes,OmniFlix Network,GATA HUB,hydrogen18,Coinage x DAIC,Stakecito,Meria,BronBro,Cypher Core,ChainLayer,Active Nodes,DACM,strangelove,ushakov,c29r3,Allnodes,PoS Node,Tedcryptoio TedLotto,kooltek68,CroutonDigital,coaction,EZ Staking,Stakin,notional,WeStaking,SmartNodes Slash Protected,Yurbason,Neta DAO,Chandra Station,Stake Frites,Decloud Nodes Lab,HashKey Cloud,chainvibes,Consensus One,Stakewollecom Autocompound,Wetez,Architect Nodes,HEXXAGON,DECOMISSIONED AUDITone,Army IDs,Bit Cat,Easy 2 Stake,Nodeasycom,jetlife,Bliss Dynamics,Sikka,PLEASE REDELEGATECLOSED,SNZPool,Autonomy Formerly PrithviDevs,Baset,Huobi,ChainodeTech,kano,Cosmic Validator Auto Compound,ValidatorNode,UNDELEGATE FROM HERE,stakezone,gunray,Bi23,Psycho Node,Bitoven,Komichaincom,BoxedCloudnet,MANTRA,eol,cosmosrescue,SkyNet Validators,Cosmonaut Stakes AKT,Dwebnet,DLINODESCOM,GlacierLuo,Domination Finance,Arcturian Tech,Akash Af,ALTER pathrocknetwork,decommissioned,BridgeTower,Dora Factory PGS,MedasDigital,STAKINOLD,Cloudmos,Former Moultrie Audits Inactive,MOVING TO NAKANODOXYZ,VaultStaking,TRGC By Liquify,astravoyage,Anonstake,polkachucom,kavalabs,AGE Network,acheron,Octopos,shepaintedfireacrossthesky,monolith,SHEA Staking,Dorminik,UNBONDING,Inactive pls undelegate,GreenStake One,inactive,Oni Stake for BoostDrop,Praetor App,0xDEADBEEF,w3coins,Vixello,Aurora Staking,50 Validator Airdrop,Passage,NoDumpNode,Jormungand 0y,INACTIVE Please Redelegate Trivium TriviumNetwork,StakeSeeker by BTCS,Crypto Lab Tech,europlots,CryptoCrew Validators,Quasarch,Ariel Akash Insider Powered by NextNetWorks,Atomstaking,black lodge,SHUTTING DOWN,Crypto and Coffee,FoundryUSA,OGnome,Defi Dojo Validation,high_similarity_voters
0,Chainflow,20,1,0,4,0,25,25,50,20,21,0.800000,0.840000,0.50,1.00,0.24,0.28,0.32,0.28,0.12,0.20,0.30,0.06,0.38,0.22,0.34,0.14,0.40,0.12,0.00,0.28,0.0,0.00,0.12,0.28,0.36,0.30,0.38,0.36,0.30,0.28,0.26,0.34,0.06,0.40,0.18,0.22,0.26,0.02,0.14,0.36,0.36,0.0,0.36,0.28,0.36,0.18,0.20,0.02,0.32,0.32,0.36,0.32,0.26,0.04,0.00,0.26,0.28,0.08,0.28,0.02,0.0,0.34,0.12,0.34,0.36,0.04,0.38,0.02,0.36,0.04,0.00,0.0,0.00,0.42,0.10,0.24,0.28,0.04,0.32,0.02,0.02,0.02,0.20,0.00,0.04,0.24,0.00,0.32,0.38,0.02,0.04,0.00,0.24,0.02,0.24,0.34,0.08,0.0,0.00,0.06,0.04,0.28,0.02,0.00,0.00,0.02,0.04,0.12,0.12,0.0,0.24,0.30,0.0,0.02,0.38,0.00,0.02,0.02,0.12,0.02,0.38,0.38,0.06,0.06,0.00,0.00,0,0.00,0.00,0.20,0.00,0.06,0.00,0.00,0.20,0.36,0.00,0.28,0.16,0.22,0.32,0.00,0.32,0.22,0.00,0.10,0.20,0.34,0.00,0.34,0.12,0.34,0.04,0.0,0.16,0.22,0.02,0.04,[]
1,Imperatorco,23,4,1,21,0,49,49,50,25,28,0.510204,0.571429,0.98,0.24,1.00,0.72,0.52,0.48,0.22,0.28,0.36,0.12,0.48,0.42,0.38,0.08,0.54,0.42,0.12,0.20,0.0,0.02,0.12,0.32,0.44,0.38,0.48,0.58,0.46,0.66,0.40,0.40,0.04,0.62,0.40,0.42,0.24,0.02,0.14,0.40,0.40,0.0,0.38,0.50,0.52,0.26,0.20,0.06,0.36,0.42,0.46,0.26,0.24,0.04,0.04,0.24,0.62,0.30,0.42,0.00,0.0,0.42,0.22,0.44,0.44,0.04,0.50,0.00,0.52,0.16,0.04,0.0,0.04,0.50,0.20,0.34,0.28,0.10,0.44,0.02,0.06,0.02,0.28,0.02,0.06,0.26,0.02,0.58,0.48,0.00,0.08,0.02,0.34,0.08,0.56,0.42,0.08,0.0,0.06,0.02,0.10,0.30,0.02,0.02,0.02,0.00,0.04,0.10,0.10,0.0,0.34,0.24,0.0,0.06,0.50,0.06,0.04,0.06,0.22,0.00,0.42,0.50,0.12,0.08,0.02,0.02,0,0.02,0.02,0.14,0.02,0.24,0.02,0.02,0.16,0.44,0.00,0.48,0.14,0.18,0.42,0.04,0.46,0.26,0.02,0.12,0.16,0.36,0.08,0.34,0.04,0.32,0.06,0.0,0.10,0.14,0.00,0.04,[ AutoStake Slash Protect...
2,AutoStake Slash Protected,34,5,1,10,0,50,50,50,36,37,0.720000,0.740000,1.00,0.